In [2]:
import datetime
import requests


def gen_secid(symbol):
    exchange = symbol[0:2]
    code = symbol[2:]
    if exchange == "SH":
        return f"1.{code}"
    elif exchange == "ZS":
        f3 = code[0:3]
        if f3 == "000":
            return f"1.{code}"

    return f"0.{code}"


# 不复权
def cn_chartbar(symbol, start=None, end=None):
    if not start:
        start = "20100101"
    else:
        start = start.replace("-", "")
    if not end:
        end = datetime.datetime.now().strftime("%Y%m%d")
    else:
        end = end.replace("-", "")
    secid = gen_secid(symbol)
    #fqt 不复权 : 0; 前复权 : 1; 后复权 : 2
    url = f'https://push2his.eastmoney.com/api/qt/stock/kline/get?secid={secid}&fields1=f1%2Cf2%2Cf3%2Cf4%2Cf5' \
          f'&fields2=f51%2Cf52%2Cf53%2Cf54%2Cf55%2Cf56%2Cf57%2Cf58%2Cf59%2Cf60%2Cf61&klt=101' \
          f'&fqt=2&beg={start}&end={end}'
    req = requests.get(url, timeout=30)  # request比pandas加载快
    json = req.json()
    return json

In [3]:
symb = "SZ000001"
json = cn_chartbar(symb, "2021-01-01")

In [4]:
import pandas as pd
klines = json['data']['klines']
dt = [s.split(',') for s in klines]
ori_labels = ['date','open','close','high','low','volume','turnover','amplitude','chg_pct','chg','turnover_rate']
df = pd.DataFrame.from_records(dt, columns=ori_labels, index='date')
df['volume'] = df.volume.map(lambda x: f'{x}00')
labels = ['open','high','low','close','volume','turnover','amplitude','chg_pct','chg','turnover_rate']
df = df[labels] # 调整一下column顺序
df.to_csv("/Users/max/sz000001.csv")
# klines.insert(0,'date,open,close,high,low,volume,turnover,amplitude,chg_pct,chg,turnover_rate')
# for s in klines:
#         print(s)

# dt = [s.split(',') for s in klines]
# dt
# df = pd.DataFrame(klines, columns=['date','open','close','high','low','volume','turnover','amplitude','chg_pct','chg','turnover_rate'])
# df

In [ ]:
# type(df['turnover'][0])
ndf = df.apply(pd.to_numeric)  # convert all columns of DataFrame
# ndf

In [92]:
# type(ndf.index[0])
ndf.index = [datetime.datetime.strptime(x,'%Y-%m-%d') for x in ndf.index] # 批量修改列数据类型
# ndf.index

In [105]:
import backtrader as bt

class TestStrategy(bt.Strategy):
    def __init__(self) -> None:
        self.sma5 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=5)
        self.sma10 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=10)
    def log(self, txt, dt=None):
            ''' 日志函数，用于统一输出日志格式 '''
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))
    def next(self):
        # 记录收盘价
        self.log('Close, %.2f' % self.data.close)

        # 是否正在下单，如果是的话不能提交第二次订单
        if self.order:
            return

        # 是否已经买入
        if not self.position:
            # 还没买，如果 MA5 > MA10 说明涨势，买入
            if self.sma5[0] > self.sma10[0]:
                self.log('BUY CREATE, %.2f' % self.data.lose)
                self.order = self.buy()

        else:
            # 已经买了，如果 MA5 < MA10 ，说明跌势，卖出
            if self.sma5[0] < self.sma10[0]:
                self.log('SELL CREATE, %.2f' % self.data.close)
                self.order = self.sell()

In [120]:
import backtrader as bt
class MyStrategy(bt.Strategy):

    def __init__(self):
        self.sma = bt.ind.SimpleMovingAverage(period=15)

    def next(self):
        if self.sma > self.data.close:
            # Do something
            print("sma>close: ", self.data.close[0])

        elif self.sma < self.data.close:
            # Do something else
            print("sma<close: ", self.data.close[0])

In [ ]:
# %matplotlib
import backtrader as bt
cerebro = bt.Cerebro()
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(0.005)
cerebro.addsizer(bt.sizers.FixedSize, stake=100)
cerebro.addstrategy(MyStrategy)

# Pass it to the backtrader datafeed and add it to the cerebro
data = bt.feeds.PandasData(dataname=ndf)
cerebro.adddata(data)
cerebro.run()
cerebro.plot() # 无图???

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import backtrader as bt

class St(bt.Strategy):
    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data)

data = bt.feeds.BacktraderCSVData(dataname='/Users/max/sz000001.csv')
cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(St)
cerebro.run()
cerebro.plot(style='bar')

In [ ]:
# dt['data'] #['klines']
# sh = cn_chartbar('SH000001')
# sh  #7775
# sz = cn_chartbar('SZ399001')
# sz    #7728
# cy = cn_chartbar('SZ399006')
# cy    #3007